In [1]:
# import libraries
import pandas as pd
import requests
import xml.dom.minidom as m
import xml.etree.ElementTree as et
import time as time

In [6]:
# retrieve individual article data from PubMed
# used as helper method in later functions
# modifying code given in assignment document
# def get_info(url):
#     r = requests.get(f"{url}")
#     doc = m.parseString(r.text)
#     titles = doc.getElementsByTagName("Title")
#     abstracts = doc.getElementsByTagName("AbstractText")
#     title = titles[0].childNodes[0].wholeText
#     abstract = abstracts[0].childNodes[0].wholeText
#     return title, abstract

In [5]:
# get id list on Alzheimer search
# url for Alzheimer search: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=+AND+2019[pdat]&retmax=1000&retmode=xml
al_xtree = et.parse("alzheimer_esearch.xml")
al_xroot = al_xtree.getroot()

ids = []

# retrieve 1000 article ids dated 2019 regarding Alzheimers on PubMed
for node in al_xroot[3]:
    ids.append(str(node.attrib.get("Id")))

print("There are", len(ids), "ids extracted")

There are 1000 ids extracted


In [ ]:
# getting articles identified by ids through POST
r = requests.post("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Alzheimers+AND+2019[pdat]&retmax=1000&retmode=xml")

In [ ]:
# general url for locating each paper using pmid: url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&id="+id+"&retmode=xml"

col_name = ["title", "abstract", "query", "mesh"]
